# Домашняя работа часть 2  (10 баллов): совмещение спарсификации и квантизации.

В это задании мы предлагаем вам самим реализовать функции квантизации и спарсисикации и далее совместить их вместе. В качестве базовых методов предлагаем взять Magnitude Pruning и симметричную поканальную RTN квантизацию (коэффициент квантизации выбирается для каждого столбца). Также возможны альтерантивные варианты. Например, можно объединить Wanda Pruning и RTN, GPTQ и SparseGPT, или использование других методов.

Результат работы - это метрики L1 и L2  для $|C(\mathbf{W})\mathbf{X}^T - \mathbf{W}\mathbf{X}^T|$ для каждого слоя сжатого в два раза, количество различных значений весов в каждом столбце матрицы $C(\mathbf{W})$ и общее количество нулевых значений в этой матрице, где $C$  - функция одновременной спарсификации и квантизации. При этом считаем, что понижение битовости в два раза соотвествует сжатию в два раза, зануление половины весов так же соответствует сжатию в два раза.

Чтобы упросить задание, мы не будем квантизовать всю модель, а квантизуем только некоторые слои.

В папке llama7b_weights содержатся веса для одоного слоя LLaMа7b и сопутствующие активации в папке llama7b_act_scales (аггрегировые по датасету)

В качестве результата нужно будет сдать ноутбук с решением и метриками.

### Баллы

Релизация алгортима спарсификации - 2 балла <br>
Релизация алгортима квантизации - 2 балла <br>
Объеденение двух алгоритмов - 6 баллов <br>

In [1]:
!pip uninstall gdown -y && pip install gdown
!gdown -V

Found existing installation: gdown 5.2.0
Uninstalling gdown-5.2.0:
  Successfully uninstalled gdown-5.2.0
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
gdown 5.2.0 at /usr/local/lib/python3.10/dist-packages


In [2]:
import gdown
#Загрузка весов модели
url = 'https://drive.google.com/uc?export=download&id=1n2zThYW1MXqIIPdmQTn1-ANSDhSpLArx'
output = 'llama7b_weights.zip'
gdown.download(url, output, quiet=False)
!unzip llama7b_weights.zip

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1n2zThYW1MXqIIPdmQTn1-ANSDhSpLArx
From (redirected): https://drive.google.com/uc?export=download&id=1n2zThYW1MXqIIPdmQTn1-ANSDhSpLArx&confirm=t&uuid=af1ffd40-7aa4-41af-a73b-5a28d79e5543
To: /content/llama7b_weights.zip
100%|██████████| 311M/311M [00:07<00:00, 44.0MB/s]


Archive:  llama7b_weights.zip
replace llama7b_weights/model.layers.20.mlp.down_proj.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: llama7b_weights/model.layers.20.mlp.down_proj.pt  
  inflating: llama7b_weights/model.layers.20.self_attn.v_proj.pt  
  inflating: llama7b_weights/model.layers.20.mlp.gate_proj.pt  
  inflating: llama7b_weights/model.layers.20.self_attn.k_proj.pt  
  inflating: llama7b_weights/model.layers.20.mlp.up_proj.pt  
  inflating: llama7b_weights/model.layers.20.self_attn.q_proj.pt  
  inflating: llama7b_weights/model.layers.20.self_attn.o_proj.pt  


In [3]:
#Загрузка усредненных активаций
url = 'https://drive.google.com/uc?export=download&id=1VnJbohhzLYAP4X1OAHz3gyagj-T7vOKl'
output = 'llama7b_act_scales.zip'
gdown.download(url, output, quiet=False)
!unzip llama7b_act_scales.zip

Downloading...
From: https://drive.google.com/uc?export=download&id=1VnJbohhzLYAP4X1OAHz3gyagj-T7vOKl
To: /content/llama7b_act_scales.zip
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.0MB/s]


Archive:  llama7b_act_scales.zip
replace llama7b_act_scales/Llama-2-7b-hf.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: llama7b_act_scales/Llama-2-7b-hf.pt  


In [4]:
import os
import torch
from tqdm import tqdm
from functools import partial

In [5]:
# load weights

FOLDER = "llama7b_weights/"
names = os.listdir(FOLDER)
weight_paths =  {name.replace(".pt",'') :os.path.join(FOLDER, name) for name in names}
names = [name.replace(".pt",'') for name in names]
W = torch.load(weight_paths[names[1]])
W.shape

<ipython-input-5-5752a9a2f6fe>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  W = torch.load(weight_paths[names[1]])


torch.Size([4096, 11008])

In [6]:
torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")['model.layers.0.self_attn.q_proj'].shape

<ipython-input-6-5c754a4e4ef8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")['model.layers.0.self_attn.q_proj'].shape


torch.Size([4096])

In [92]:
# Пример расчета матрицы Гессе H для выбранного слоя,
#которая пригодится если реализовывать GPTQ, SparseGPT
X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")[names[1]]

H = torch.outer(X, X)

damp = 0.01 * torch.mean(torch.diag(H))
diag = torch.arange(X.shape[0])
H[diag, diag] += damp

H = torch.linalg.cholesky(H)
H = torch.cholesky_inverse(H)
H = torch.linalg.cholesky(H, upper=True)
Hinv = H
H.shape

In [57]:
X.shape

torch.Size([11008])

Для вдохновения рекомендуем воспользоваться кодом QUIK

https://github.com/IST-DASLab/QUIK/blob/9558d7121c698174fc93940e0b52c38c746c97ea/experiments/quik_utils.py#L80

# Ваше решение тут ...

In [71]:
def magnitude_pruning(tensor, *args, prune_rate=0.25):
  assert 0 < prune_rate <= 1
  prune_count = int(tensor.numel() * prune_rate)

  new_tensor = tensor.clone()
  thrs = torch.topk(tensor.abs().flatten(), prune_count, largest=False).values.max()
  mask = new_tensor.abs() > thrs
  return new_tensor * mask, prune_rate * 100

In [21]:
def _compute_Hinv(X):
  H = torch.outer(X, X)

  damp = 0.01 * torch.mean(torch.diag(H))
  diag = torch.arange(X.shape[0])
  H[diag, diag] += damp

  H = torch.linalg.cholesky(H)
  H = torch.cholesky_inverse(H)
  H = torch.linalg.cholesky(H, upper=True)
  Hinv = H
  return Hinv

def _quant_tensor(tensor, bitwidth):
  max_q = 2**(bitwidth - 1) - 1
  min_q = -2**(bitwidth - 1)
  scale = (tensor.max() - tensor.min()) / (max_q - min_q)
  return scale * torch.clip(tensor / scale, min=min_q, max=max_q).long()


def gpqt_quantization(W_init, X, block_size=128, bitwidth=4):
  W = W_init.clone()
  Hinv = _compute_Hinv(X)
  Q = torch.zeros_like(W)

  n_cols = W.shape[1]
  for i1 in range(0, n_cols, block_size):
    i2 = min(i1 + block_size, n_cols)

    W_block = W[:, i1:i2].clone()
    Q_block = torch.zeros_like(W_block)
    Err_block = torch.zeros_like(W_block)
    Hinv_block = Hinv[i1:i2, i1:i2]

    for j in range(i2 - i1):
        w_j = W_block[:, j]
        hinv_j = Hinv_block[j, j]

        # quant
        w_j_quant = _quant_tensor(w_j, bitwidth).flatten()

        Q_block[:, j] = w_j_quant

        err_j = (w_j - w_j_quant) / hinv_j
        Err_block[:, j] = err_j
        W_block[:, j:] -= err_j.unsqueeze(1).matmul(Hinv_block[j, j:].unsqueeze(0))

    Q[:, i1:i2] = Q_block

    W[:, i2:] -= Err_block.matmul(Hinv[i1:i2, i2:])

  return Q, bitwidth

In [82]:
def magnitude_pruning_and_gpqt_quantization(W_init, X, block_size=128, bitwidth=4, prune_rate=0.25):
  W = W_init.clone()
  Hinv = _compute_Hinv(X)
  Q = torch.zeros_like(W)

  n_cols = W.shape[1]
  for i1 in range(0, n_cols, block_size):
    i2 = min(i1 + block_size, n_cols)

    W_block = W[:, i1:i2].clone()
    Q_block = torch.zeros_like(W_block)
    Err_block = torch.zeros_like(W_block)
    Hinv_block = Hinv[i1:i2, i1:i2]

    for j in range(i2 - i1):
        w_j = W_block[:, j]
        hinv_j = Hinv_block[j, j]

        # quant
        w_j_quant = _quant_tensor(w_j, bitwidth).flatten()

        Q_block[:, j] = w_j_quant

        err_j = (w_j - w_j_quant) / hinv_j
        Err_block[:, j] = err_j
        W_block[:, j:] -= err_j.unsqueeze(1).matmul(Hinv_block[j, j:].unsqueeze(0))

    Q_block_prunned, _ = magnitude_pruning(Q_block, prune_rate=prune_rate)
    Q[:, i1:i2] = Q_block_prunned

    W[:, i2:] -= Err_block.matmul(Hinv[i1:i2, i2:])

  return Q, bitwidth * (1 / prune_rate)

#### Пример расчета ошибки

In [22]:
#  Фкнкцию ниже можно переписать для своего удобства

def test(your_quantization_function):
    c = your_quantization_function
    result = dict()
    for name in tqdm(names):
        X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")[name]
        W = torch.load(weight_paths[name])
        X = X.float()
        W = W.float()
        WQ, compress_size = c(W, X)
        l2 = ((W@X.T - WQ@X.T)**2).mean()
        l1 = (W@X.T - WQ@X.T).abs().mean()
        result[name] = (l2,l1, compress_size)
    return  result


def dummy_compress(X, *args):
    X = torch.round(X)
    compress_size = 0
    return X, compress_size


test(dummy_compress)

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(3.5155), tensor(1.4972), 0),
 'model.layers.20.mlp.down_proj': (tensor(158.4233), tensor(10.0173), 0),
 'model.layers.20.mlp.gate_proj': (tensor(4.2363), tensor(1.6319), 0),
 'model.layers.20.self_attn.v_proj': (tensor(4.8678), tensor(1.7560), 0),
 'model.layers.20.self_attn.o_proj': (tensor(4.2832), tensor(1.6623), 0),
 'model.layers.20.self_attn.k_proj': (tensor(11.6558), tensor(2.6132), 0),
 'model.layers.20.self_attn.q_proj': (tensor(10.3696), tensor(2.4367), 0)}

### Prunning

In [72]:
test(partial(magnitude_pruning, prune_rate=0.5)) # prune_rate x2

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(0.2558), tensor(0.4042), 50.0),
 'model.layers.20.mlp.down_proj': (tensor(10.7266), tensor(2.6048), 50.0),
 'model.layers.20.mlp.gate_proj': (tensor(0.2858), tensor(0.4271), 50.0),
 'model.layers.20.self_attn.v_proj': (tensor(0.3829), tensor(0.4923), 50.0),
 'model.layers.20.self_attn.o_proj': (tensor(0.2842), tensor(0.4236), 50.0),
 'model.layers.20.self_attn.k_proj': (tensor(0.5403), tensor(0.5837), 50.0),
 'model.layers.20.self_attn.q_proj': (tensor(0.5276), tensor(0.5807), 50.0)}

In [74]:
test(partial(magnitude_pruning, prune_rate=0.75)) # prune_rate x4

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(0.9916), tensor(0.8000), 75.0),
 'model.layers.20.mlp.down_proj': (tensor(40.9887), tensor(5.1487), 75.0),
 'model.layers.20.mlp.gate_proj': (tensor(1.1512), tensor(0.8521), 75.0),
 'model.layers.20.self_attn.v_proj': (tensor(1.3904), tensor(0.9349), 75.0),
 'model.layers.20.self_attn.o_proj': (tensor(1.1469), tensor(0.8606), 75.0),
 'model.layers.20.self_attn.k_proj': (tensor(2.1815), tensor(1.1675), 75.0),
 'model.layers.20.self_attn.q_proj': (tensor(2.0661), tensor(1.1415), 75.0)}

### Quantization

In [75]:
test(partial(gpqt_quantization, bitwidth=8)) # int8

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(1.8132e-07), tensor(0.0004), 8),
 'model.layers.20.mlp.down_proj': (tensor(8.1609e-07), tensor(0.0008), 8),
 'model.layers.20.mlp.gate_proj': (tensor(3.6551e-07), tensor(0.0005), 8),
 'model.layers.20.self_attn.v_proj': (tensor(2.4336e-07), tensor(0.0004), 8),
 'model.layers.20.self_attn.o_proj': (tensor(3.3170e-07), tensor(0.0005), 8),
 'model.layers.20.self_attn.k_proj': (tensor(5.5133e-07), tensor(0.0006), 8),
 'model.layers.20.self_attn.q_proj': (tensor(5.1547e-07), tensor(0.0006), 8)}

In [76]:
test(partial(gpqt_quantization, bitwidth=4)) # int4

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(6.4006e-05), tensor(0.0068), 4),
 'model.layers.20.mlp.down_proj': (tensor(0.0003), tensor(0.0145), 4),
 'model.layers.20.mlp.gate_proj': (tensor(9.8604e-05), tensor(0.0085), 4),
 'model.layers.20.self_attn.v_proj': (tensor(8.9910e-05), tensor(0.0081), 4),
 'model.layers.20.self_attn.o_proj': (tensor(0.0001), tensor(0.0099), 4),
 'model.layers.20.self_attn.k_proj': (tensor(0.0001), tensor(0.0104), 4),
 'model.layers.20.self_attn.q_proj': (tensor(0.0002), tensor(0.0110), 4)}

In [84]:
test(partial(gpqt_quantization, bitwidth=2)) # int4

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(0.0953), tensor(0.2595), 2),
 'model.layers.20.mlp.down_proj': (tensor(1.1166), tensor(0.9365), 2),
 'model.layers.20.mlp.gate_proj': (tensor(0.1179), tensor(0.2889), 2),
 'model.layers.20.self_attn.v_proj': (tensor(0.0917), tensor(0.2551), 2),
 'model.layers.20.self_attn.o_proj': (tensor(0.0887), tensor(0.2365), 2),
 'model.layers.20.self_attn.k_proj': (tensor(0.2321), tensor(0.4041), 2),
 'model.layers.20.self_attn.q_proj': (tensor(0.2009), tensor(0.3771), 2)}

## Prunning and quantization

In [83]:
test(partial(magnitude_pruning_and_gpqt_quantization, bitwidth=4, prune_rate=0.5)) # int4 and prune_rate x2

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-22-b5cca9bf838d>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./llama7b_act_scales/Llama-2-7b-hf.pt")

{'model.layers.20.mlp.up_proj': (tensor(0.0884), tensor(0.2366), 8.0),
 'model.layers.20.mlp.down_proj': (tensor(3.1419), tensor(1.4139), 8.0),
 'model.layers.20.mlp.gate_proj': (tensor(0.0465), tensor(0.1728), 8.0),
 'model.layers.20.self_attn.v_proj': (tensor(0.1512), tensor(0.3109), 8.0),
 'model.layers.20.self_attn.o_proj': (tensor(0.0917), tensor(0.2424), 8.0),
 'model.layers.20.self_attn.k_proj': (tensor(0.0389), tensor(0.1561), 8.0),
 'model.layers.20.self_attn.q_proj': (tensor(0.0540), tensor(0.1841), 8.0)}

# Вывод

- эффективнее всего работает gpqt_quantization, как в 8 бит, так и в 4 бит по сравнению с пруннигом, где сжатие уже в 2 раза (compress_size=50) уже сильно теряет в качестве
- квантизация не падает драматически в качестве даже при квантизации в 2 бит
- при этом комбинация пруннинг и квантизация работают (последний замер) почти также как и gpqt_quantization в отдельности